# Dependencies

In [64]:
model_name = "gemini-2.0-flash-001"
dataset = "product_data_mrt"
region = "us-central1"
project_id = "amazon-product-reviews-452322"
region = "us-central1"

In [65]:
from google.cloud import bigquery

bq_client = bigquery.Client(project=project_id, location=region)

dataset_id = bigquery.Dataset(f"{project_id}.{dataset}")
dataset_id.location = region
resp = bq_client.create_dataset(dataset_id, exists_ok=True)
print("Created dataset {}.{}".format(bq_client.project, resp.dataset_id))

Created dataset amazon-product-reviews-452322.product_data_mrt


# Business Questions:





**1. How do product features and verified purchases influence ratings and helpful votes?**

Understand if specific product features combined with verified purchases lead to higher ratings and more helpful votes.
Identify which features to enhance and if promoting verified purchases can boost credibility.


**2. Which product categories and membership plans lead to higher repeat purchases and ratings?**

Determine if certain product categories combined with membership plans drive more repeat purchases and higher ratings.
Guide strategies for product bundling and membership perks.

**3. How does price sensitivity vary across product categories and devices used?**



Understand if price sensitivity is higher on certain devices or in specific product categories.
Guide pricing strategies based on device type and category.

**4)Which product categories have the highest average ratings?**
Purpose:

Identify popular categories for targeted marketing and promotions.
Mart Used:

mart_category_performance
Potential Insights:

Categories with the highest ratings can be prioritized for ads or discounts.

**5) How do verified purchases impact the average rating?**
Purpose:

Determine if verified purchases tend to leave higher or lower ratings.
Mart Used:

mart_verified_purchase_impact
Potential Insights:

Verified purchases might provide more reliable feedback.
A higher average rating for verified purchases suggests credibility.

**6)What is the average review rating by device type?**
Purpose:

Understand if certain devices lead to higher or lower ratings.
Mart Used:

mart_device_ratings


Potential Insights:

Mobile users might rate differently compared to desktop users.
Can guide mobile or desktop-specific UX improvements.

# Developing Marts

## mart_category_performance

In [66]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE product_data_mrt.mart_category_performance AS
SELECT
    c.main_category,
    AVG(pmdm.average_rating) AS avg_rating,
    COUNT(pmdm.title) AS total_products,
    CURRENT_TIMESTAMP() AS _load_time
FROM product_data_int.Categories c
INNER JOIN product_data_int.Product_Meta_Data_Merged pmdm
ON c.main_category = pmdm.main_category
WHERE pmdm.average_rating IS NOT NULL
GROUP BY c.main_category;


Query is running:   0%|          |

""


## mart_device_ratings

In [67]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE product_data_mrt.mart_device_ratings AS
SELECT
    ui.devices_used AS device_type,
    AVG(pr.rating) AS average_rating,
    COUNT(u.review_id) AS total_reviews,
    CURRENT_TIMESTAMP() AS _load_time
FROM product_data_int.User_Info ui
INNER JOIN product_data_int.User u
    ON ui.user_id = u.user_id
INNER JOIN product_data_int.Product_Reviews pr
    ON u.user_id = pr.user_id
INNER JOIN product_data_int.Product_Image_Data pid
    ON pr.ASIN = pid.ASIN
GROUP BY ui.devices_used;


Query is running:   0%|          |

""


## mart_verified_purchase_impact

In [68]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE product_data_mrt.mart_verified_purchase_impact AS
SELECT
    verified_purchase,
    AVG(rating) AS average_rating,
    AVG(helpful_vote) AS average_helpful_votes,
    COUNT(review_id) AS total_reviews,
    CURRENT_TIMESTAMP() AS _load_time
FROM product_data_int.Product_Image_Data
GROUP BY verified_purchase;

Executing query with job ID: d955c822-185d-493f-bcb0-8ac0283eeba8
Query executing: 0.30s


ERROR:
 400 Unrecognized name: verified_purchase at [3:5]; reason: invalidQuery, location: query, message: Unrecognized name: verified_purchase at [3:5]

Location: us-central1
Job ID: d955c822-185d-493f-bcb0-8ac0283eeba8



# Average Rating by Subscription Plan


In [69]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE product_data_mrt.mart_avg_rating_by_plan AS
SELECT
    mi.subscription_plan,
    AVG(pr.rating) AS average_rating,
    CURRENT_TIMESTAMP() AS _load_time
FROM product_data_int.Membership_Info mi
INNER JOIN product_data_int.User u ON mi.user_id = u.user_id
INNER JOIN product_data_int.Product_Reviews pr ON u.user_id = pr.user_id
GROUP BY mi.subscription_plan;


Query is running:   0%|          |

""


# Total Reviews per Device Type

In [84]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE product_data_mrt.mart_reviews_by_device AS
SELECT
    ui.devices_used AS device_type,
    COUNT(pr.user_id) AS total_reviews,
    CURRENT_TIMESTAMP() AS _load_time
FROM product_data_int.User_Info ui
INNER JOIN product_data_int.User u ON ui.user_id = u.user_id
INNER JOIN product_data_int.Product_Reviews pr ON u.user_id = pr.user_id
GROUP BY ui.devices_used;


Query is running:   0%|          |

""


# Most Popular Genres

In [71]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE product_data_mrt.mart_popular_genres AS
SELECT
    genre,
    COUNT(*) AS count,
    CURRENT_TIMESTAMP() AS _load_time
FROM UNNEST(SPLIT(ui.favorite_genres, ',')) AS genre
GROUP BY genre
ORDER BY count DESC;


Executing query with job ID: 092fbdc8-4cb6-4b01-88f3-d9acf29112e8
Query executing: 0.23s


ERROR:
 400 Unrecognized name: ui at [6:19]; reason: invalidQuery, location: query, message: Unrecognized name: ui at [6:19]

Location: us-central1
Job ID: 092fbdc8-4cb6-4b01-88f3-d9acf29112e8



# Top Rated Products

In [72]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE product_data_mrt.mart_top_rated_products AS
SELECT
    pr.ASIN,
    AVG(pr.rating) AS average_rating,
    CURRENT_TIMESTAMP() AS _load_time
FROM product_data_int.Product_Reviews pr
GROUP BY pr.ASIN
ORDER BY average_rating DESC
LIMIT 5;


Query is running:   0%|          |

""


# Active Membership Count

In [73]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE product_data_mrt.mart_active_memberships AS
SELECT
    COUNT(*) AS active_memberships,
    CURRENT_TIMESTAMP() AS _load_time
FROM product_data_int.Membership_Info
WHERE membership_end_date IS NULL OR membership_end_date > CURRENT_DATE();


Query is running:   0%|          |

""


# Average Rating and Review Count by Device Type



In [80]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE product_data_mrt.mart_avg_rating_review_count_by_device AS
SELECT
    ui.devices_used AS device_type,
    COUNT(DISTINCT pr.user_id) AS total_reviews,
    AVG(ui.feedback_ratings) AS average_feedback_rating,
    AVG(pr.rating) AS average_rating,
    CURRENT_TIMESTAMP() AS _load_time
FROM product_data_int.User_Info ui
INNER JOIN product_data_int.User u ON ui.user_id = u.user_id
INNER JOIN product_data_int.Product_Reviews pr ON u.user_id = pr.user_id
GROUP BY ui.devices_used;


Query is running:   0%|          |

""


# Mart: Active vs. Inactive Membership Counts

In [75]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE product_data_mrt.mart_membership_status_counts AS
SELECT
    COUNT(CASE WHEN mi.membership_end_date IS NULL OR mi.membership_end_date > CURRENT_DATE() THEN 1 END) AS active_memberships,
    COUNT(CASE WHEN mi.membership_end_date <= CURRENT_DATE() THEN 1 END) AS inactive_memberships,
    CURRENT_TIMESTAMP() AS _load_time
FROM product_data_int.Membership_Info mi;


Query is running:   0%|          |

""


# User Info Based Metrics

In [76]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE product_data_mrt.mart_user_info_based_metrics AS
SELECT
    -- Device-based Metrics
    ui.devices_used AS device_type,
    COUNT(DISTINCT pr.user_id) AS total_reviews,
    AVG(pr.rating) AS average_rating,

    -- Feedback and Support Metrics
    AVG(ui.feedback_ratings) AS average_feedback,
    COUNT(CASE WHEN ui.customer_support_interactions > 5 THEN 1 END) AS high_interaction_users,

    -- Usage Frequency Metrics
    ui.usage_frequency,
    COUNT(ui.user_id) AS user_count_by_frequency,

    -- Genre-based Metrics
    genre,
    AVG(pr.rating) AS average_rating_by_genre,
    COUNT(DISTINCT ui.user_id) AS genre_user_count,

    CURRENT_TIMESTAMP() AS _load_time
FROM product_data_int.User_Info ui
INNER JOIN product_data_int.User u ON ui.user_id = u.user_id
INNER JOIN product_data_int.Product_Reviews pr ON u.user_id = pr.user_id
CROSS JOIN UNNEST(SPLIT(ui.favorite_genres, ',')) AS genre
GROUP BY
    ui.devices_used,
    ui.usage_frequency,
    genre;


Query is running:   0%|          |

""


In [79]:
%%bigquery --project amazon-product-reviews-452322

SELECT
    -- Columns from User_Info
    ui.user_id,
    ui.usage_frequency,
    ui.purchase_history,
    ui.favorite_genres,
    ui.devices_used,
    ui.engagement_metrics,
    ui.feedback_ratings,
    ui.customer_support_interactions,
    ui._data_source AS ui_data_source,
    ui._load_time AS ui_load_time,

    -- Columns from User
    u.username,
    ur.review_id AS user_review_id,

    -- Columns from Product_Reviews
    pr.rating,
    pr.title,
    pr.ASIN,
    pr.review_date,

    -- Columns from Membership_Info
    mi.subscription_plan,
    mi.membership_start_date,
    mi.membership_end_date,

    CURRENT_TIMESTAMP() AS _load_time
FROM product_data_int.User_Info ui
INNER JOIN product_data_int.User u ON ui.user_id = u.user_id
INNER JOIN product_data_int.User_Reviews ur ON u.user_id = ur.user_id
INNER JOIN product_data_int.Product_Reviews pr ON ur.user_id = pr.user_id
INNER JOIN product_data_int.Product_Image_Data pid ON pr.ASIN = pid.ASIN
INNER JOIN product_data_int.Membership_Info mi ON u.user_id = mi.user_id
LIMIT 20;


Query is running:   0%|          |

Downloading: |          |

,user_id,usage_frequency,purchase_history,favorite_genres,devices_used,engagement_metrics,feedback_ratings,customer_support_interactions,ui_data_source,ui_load_time,username,user_review_id,rating,title,ASIN,review_date,subscription_plan,membership_start_date,membership_end_date,_load_time


# Subscription Plan Distribution

In [81]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE product_data_mrt.mart_subscription_plan_distribution AS
SELECT
    mi.subscription_plan,
    COUNT(DISTINCT mi.user_id) AS user_count,
    CURRENT_TIMESTAMP() AS _load_time
FROM product_data_int.Membership_Info mi
GROUP BY mi.subscription_plan;


Query is running:   0%|          |

""


# Product Count by Brand and Category


In [83]:
%%bigquery --project amazon-product-reviews-452322

CREATE OR REPLACE TABLE product_data_mrt.mart_product_count_by_brand_category AS
SELECT
    brand_name,
    main_category,
    COUNT(DISTINCT product_id) AS product_count,
    CURRENT_TIMESTAMP() AS _load_time
FROM product_data_int.Product_Image_Data
GROUP BY brand_name, main_category;


Query is running:   0%|          |

""
